# Econometria Aplicada

## Aula 5 - Modelos com dados em painel




João Ricardo Costa Filho \\
[joaocostafilho.com](https://)

**Motivação**: \\

 Qual a relação entre o investimento das empresas com o seu estoque de capital e o valor de mercado das mesmas? \
 Esta aplicação foi inspirada no tutorial ["R Tutorial: Panel Data Analysis 1"](https://rpubs.com/phle/r_tutorial_panel_data_analysis) e utiliza dados de Grunfeld (1958). \
 \

 Grunfeld, Y. (1958). The Determinants of Corporate Investment. Unpublished Ph.D. Dissertation, University of Chicago

### Pacotes

In [ ]:
 install.packages("plm") # pacote para estimar modelos com dados em painel

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘miscTools’, ‘rbibutils’, ‘bdsmatrix’, ‘collapse’, ‘zoo’, ‘sandwich’, ‘lmtest’, ‘maxLik’, ‘Rdpack’, ‘Formula’




In [ ]:
library(plm) 

### Dados

In [ ]:
data("Grunfeld", package="plm")

In [ ]:
?Grunfeld

Veja os primeiros 30 elementos da base de dados:

Veja a estrutura dos dados:

Variáveis:

* **invest**: "Gross investment, defined as additions to plant and equipment plus maintenance and repairs in millions of dollars deflated by the implicit price deflator of producers’ durable equipment (base 1947)."
* **value**: "Market value of the firm, defined as the price of common shares at December 31 (or, for WH, IBM and CH, the average price of December 31 and January 31 of the following year) times the number of common shares outstanding plus price of preferred shares at December 31 (or average price of December 31 and January 31 of the following year) times number of preferred shares plus total book value of debt at December 31 in millions of dollars deflated by the implicit GNP price deflator (base 1947)."
* **capital**: "Stock of plant and equipment, defined as the accumulated sum of net additions to plant and equipment deflated by the implicit price deflator for producers’ durable equipment (base 1947) minus depreciation allowance deflated by depreciation expense deflator (10 years moving average of wholesale price index of metals and metal products, base 1947)."
* **firm**: "factor with 11 levels: "General Motors", "US Steel", "General Electric", "Chrysler", "Atlantic Refining", "IBM", "Union Oil", "Westinghouse", "Goodyear", "Diamond Match", "American Steel"."
* **year**: "Year."

Fonte: https://cran.radicaldevelop.com/web/packages/AER/AER.pdf

## Visualização dos dados

Faça um gráfico de dispersão entre o investimento e o estoque de capital

In [ ]:
library(ggplot2)

Faça um gráfico do investimento por empresa ao longo do tempo:

In [ ]:
ggplot(data = Grunfeld, aes(x = year, y = inv) ) +
  geom_line( aes( colour = as.factor(firm) ) ) +
  labs(x = "Ano",  y = "Investimento") +
  theme_classic() +
  theme(plot.title = element_text(size = 18, face = "bold"),
        axis.title.x = element_text(size = 14, face = "bold"),
        axis.title.y = element_text(size = 14, face = "bold")
        ) +
  theme(panel.background = element_rect(fill = "#f0f0f0"),
        plot.background = element_rect(fill = "#f0f0f0"),
        panel.grid.major = element_line(colour = "white"),
        legend.position = "none" )

## Modelo de regressão linear

Vamos estimar a regressão abaixo:

\begin{equation}
\text{ inv}_i = \beta_0 + \beta_1 \text{ capital}_i + \beta_2 \text{ value}_i + \epsilon_i
\end{equation}



In [ ]:
install.packages("broom")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(broom) 

Alternativa com o pacote 'plm':

In [ ]:
pooled_ols_plm <- plm(inv ~ capital + value, data = Grunfeld, 
                      index = c("firm", "year"), 
                      effect = "individual", model = "pooling")

In [ ]:
results <- tidy( pooled_ols_plm )

In [ ]:
print( results )

Mas como sabemos que as mesmas firmas são acompanhadas ao longo do tempo, precisamos ajustar a matriz de variância-covariância para qualquer inferência sobre os coeficientes:

In [ ]:
vcov_cluster <- vcovHC(pooled_ols_plm, cluster = "group")

In [ ]:
pooled_ols_plm$vcov <- vcov_cluster

In [ ]:
results <- tidy( pooled_ols_plm )

In [ ]:
print( results )

Qual é a conclusão que obtemos?

### Teste para componentes não-observáveis

Teste Breusch-Pagan:

$\mathcal{H}_0: \sigma^2_u = 0$ \\
$\mathcal{H}_a: \sigma^2_u \neq 0$ \\

In [ ]:
plmtest( pooled_ols_plm, effect = "individual", 
         type = c("bp")  # Teste Breusch-Pagan                   
         )

Qual é a conclusão?

## Efeitos Fixos

#### Least Squares Dummy Variable



Vamos estimar a regressão abaixo:

\begin{equation}
\text{ inv}_{i,t} = \beta_0 + \beta_1 \text{ capital}_{i,t} + \beta_2 \text{ value}_{i,t} + u_i + \epsilon_i
\end{equation}

Primeiro, vamos estimar uma regressão com a função 'lm', mas vamos adicionar à regressão a expressão 'factor(firm)'

Vamos repetir a regressão, mas vamos adicionar '-1' à regressão para removermos o intercepto:

### Estimador de efeitos fixos

Vamos estimar o modelo com efeitos fixos, considerando a estrutura de dados em painel. Para isso, vamos utilizar a função 'plm' (como fizemos acima no caso do "pooled OLS", mas agora chamaremos o modelo de 'within': Após a estimativa, utilize o comando 'summary' para obter os resultados.

Vamos testar se a adição de um componente sistemático gera efeitos significativos:

In [ ]:
pFtest( fe_model_plm, pooled_ols_plm ) #o primeiro deve ser o modelo com efeitos fixos e o segundo o pooled OLS

### Efeitos aleatórios

Vamos estimar o modelo com efeitos aleatórios, considerando a estrutura de dados em painel. Para isso, vamos utilizar a função 'plm' (como fizemos acima, tanto no caso do "pooled OLS", como com os "efeitos fixos", mas agora chamaremos o modelo de 'random': Após a estimativa, utilize o comando 'summary' para obter os resultados.

Efeitos fixos ou aleatórios?

Teste de Hausman:

$\mathcal{H}_0: \rho \big(u, \epsilon \big) = 0$ \\
$\mathcal{H}_a: \rho \big(u, \epsilon \big) \neq 0$ \\

In [ ]:
phtest( fe_model_plm, re_model_plm )

O que podemos concluir?